In [5]:
import pandas as pd
import os
from IPython.display import display as dp

In [3]:
ventas= pd.read_csv(os.path.join('DataDE','ventas_fordh.csv'))

In [4]:
clientes= pd.read_csv(os.path.join('DataDE','clientes_fordh.csv'))

In [6]:
dp(ventas.head())

,Fecha,Producto,Categoría,Precio,Costo,Cantidad,Sucursal,ClienteID
0,2025-01-20,Alternador,Eléctrico,4500,2800,2,Puebla,251
1,2024-09-13,Amortiguador,Suspensión,900,600,4,CDMX,164
2,2025-02-19,Radiador,Enfriamiento,3500,2100,3,CDMX,45
3,2024-08-15,Filtro de Aire,Mantenimiento,300,150,2,Puebla,203
4,2024-12-08,Faros LED,Iluminación,1800,1100,2,Querétaro,27


In [8]:
ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Fecha      1000 non-null   object
 1   Producto   1000 non-null   object
 2   Categoría  1000 non-null   object
 3   Precio     1000 non-null   int64 
 4   Costo      1000 non-null   int64 
 5   Cantidad   1000 non-null   int64 
 6   Sucursal   1000 non-null   object
 7   ClienteID  1000 non-null   int64 
dtypes: int64(4), object(4)
memory usage: 62.6+ KB


In [9]:
dp(clientes.head())

,ClienteID,Nombre,TipoCliente,Ciudad,AntigüedadCliente,VolumenComprasEstimado
0,1,"Brooks, Khan and Mcdaniel",Mayorista,Bentonchester,14,436458
1,2,Woods LLC,Taller,Lake Philipview,8,318597
2,3,Nielsen-Jackson,Mayorista,South Scott,1,447031
3,4,"Mccarty, Simmons and Suarez",Distribuidor,Aprilview,11,63603
4,5,"Adams, Miller and Malone",Particular,Lake Markbury,2,332257


In [12]:
ventas['Fecha']=pd.to_datetime(ventas['Fecha'])
ventas['Mes']=ventas['Fecha'].dt.to_period('M').astype('str')

In [11]:
ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Fecha      1000 non-null   datetime64[ns]
 1   Producto   1000 non-null   object        
 2   Categoría  1000 non-null   object        
 3   Precio     1000 non-null   int64         
 4   Costo      1000 non-null   int64         
 5   Cantidad   1000 non-null   int64         
 6   Sucursal   1000 non-null   object        
 7   ClienteID  1000 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 62.6+ KB


In [13]:
ventas['Ingreso']=ventas['Precio']*ventas['Cantidad']
ventas['Margen']= (ventas['Precio']-ventas['Costo'])*ventas['Cantidad']

In [14]:
dp(ventas.head())

,Fecha,Producto,Categoría,Precio,Costo,Cantidad,Sucursal,ClienteID,Mes,Ingreso,Margen
0,2025-01-20,Alternador,Eléctrico,4500,2800,2,Puebla,251,2025-01,9000,3400
1,2024-09-13,Amortiguador,Suspensión,900,600,4,CDMX,164,2024-09,3600,1200
2,2025-02-19,Radiador,Enfriamiento,3500,2100,3,CDMX,45,2025-02,10500,4200
3,2024-08-15,Filtro de Aire,Mantenimiento,300,150,2,Puebla,203,2024-08,600,300
4,2024-12-08,Faros LED,Iluminación,1800,1100,2,Querétaro,27,2024-12,3600,1400


In [18]:
kpis=[]
for mes,grupo in ventas.groupby('Mes'):
    ingreso= grupo['Ingreso'].sum()
    margen=grupo['Margen'].sum()
    top_producto=grupo.groupby('Producto')['Cantidad'].sum().idxmax()
    sucursal_lider=grupo.groupby('Sucursal')['Ingreso'].sum().idxmax()
    categoria_top=grupo.groupby('Categoría')['Ingreso'].sum().idxmax()
    fusion=grupo.merge(clientes, on='ClienteID')
    top_cliente=fusion['TipoCliente'].value_counts().idxmax()

    kpis.append({
        'Mes':mes,
        'IngresoTotal':round(float(ingreso),2),
        'MargenTotal':round(float(margen),2),
        'TopProducto':top_producto,
        'SucursalLider':sucursal_lider,
        'CategoriaTop':categoria_top,
        'TipoClienteTop':top_cliente
    })

In [19]:
df_kpis=pd.DataFrame(kpis)

In [20]:
dp(df_kpis.head())


,Mes,IngresoTotal,MargenTotal,TopProducto,SucursalLider,CategoriaTop,TipoClienteTop
0,2024-06,283100.0,109880.0,Radiador,Puebla,Eléctrico,Mayorista
1,2024-07,300150.0,117750.0,Sensor de Oxígeno,Guadalajara,Eléctrico,Distribuidor
2,2024-08,262950.0,103010.0,Filtro de Aceite,CDMX,Eléctrico,Particular
3,2024-09,419950.0,161920.0,Batería 12V,Monterrey,Eléctrico,Distribuidor
4,2024-10,374650.0,147620.0,Pastillas de Freno,CDMX,Eléctrico,Particular


In [23]:
os.makedirs('DataDA',exist_ok=True)
df_kpis.to_csv(os.path.join('DataDA','analisis_kpis_mensuales.csv'),index=False)